In [32]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from PIL import Image
import shutil
import torch
import torch.optim as optim
from Callbacks import EarlyStopping
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

from caps_cropped_model import CapsuleNetwork, CapsuleLoss, display_images

from caps_utils import CAPS_Productive_Dataset, display_data_loader_batch, TransformDataset
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.57  Python-3.12.3 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Setup complete  (12 CPUs, 31.9 GB RAM, 343.7/397.5 GB disk)


## YOLO1

### Load Dataset YOLO1

In [33]:
image_folder="D:/Datasets/YOLO_1_Dataset/test/images"
label_folder="D:/Datasets/YOLO_1_Dataset/test/labels"

test_images= [f for f in os.listdir(image_folder) if f.endswith((".jpeg", ".JPEG"))]
test_labels= [f for f in os.listdir(label_folder) if f.endswith((".txt"))]
print(f"Gefundene testbilder: {len(test_images)}")
print(f"Gefundene testlabels: {len(test_labels)}")
print(os.path.splitext(test_images[0])[0])
image_bbox_pairs = [
                    (os.path.join(image_folder, img), os.path.join(label_folder,img.replace(".jpeg",".txt").replace(".JPEG",".txt")))
                    for img in  test_images
                    if img.replace(".jpeg",".txt").replace(".JPEG",".txt") in test_labels
]
print(f"Gefundene Paare: {len(image_bbox_pairs)}")


Gefundene testbilder: 5370
Gefundene testlabels: 5370
1.3.6.1.4.1.14519.5.2.1.6279.6001.104507274032170320323347152411
Gefundene Paare: 5370


In [34]:
def load_ground_truth(file_path, image_shape):
    h,w = image_shape[:2]
    boxes=[]

    with open(file_path, "r") as f:
        for line in f.readlines():
            class_id,x_center, y_center, width, height = map(float, line.strip().split())

            x_min = int((x_center - width/2)*w)
            y_min = int((y_center - width/2)*h)
            x_max = int((x_center + width/2)*w)
            y_max = int((y_center + width/2)*h)
        
            boxes.append([x_min,y_min,x_max,y_max])
    return boxes

### Predict Bounding Boxes with YOLO1

In [35]:
model=YOLO("YOLO1/1/weights/best.pt")
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

In [36]:
predictions_yolo1={}
ground_truths={}

for image_path, bbox_path in image_bbox_pairs:
    image = Image.open(image_path)

    image_np = np.array(image)

    gt_boxes = load_ground_truth(bbox_path, image_np.shape)
    ground_truths[os.path.basename(image_path)] = gt_boxes

    results = model(image_np)

    boxes = results[0].boxes.xyxy.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    predictions_yolo1[os.path.basename(image_path)]= {
        "boxes": boxes,
        "scores": scores,
        "classes": classes
    }


0: 512x512 (no detections), 80.6ms
Speed: 2.0ms preprocess, 80.6ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 10.0ms
Speed: 0.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.1ms
Speed: 1.9ms preprocess, 9.1ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 

In [37]:
predictions_yolo1

{'1.3.6.1.4.1.14519.5.2.1.6279.6001.104507274032170320323347152411.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.104863864417205674007098486529.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.108828997377593078753121028952.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.109466173635014704200062503592.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.109626634592628652877975591041.jpeg': {'boxes': array([], shape=(0, 4), dtype=float32),
  'scores': array([], dtype=float32),
  'classes': array([], dtype=f

In [38]:
# Funktion, um die Daten zu bereinigen
def keep_highest_score(data):
    new_data = {}
    for key, value in data.items():
        # Überprüfen, ob Scores vorhanden sind
        if value['scores'].size > 0:
            # Finde den Index des höchsten Scores
            highest_score_idx = np.argmax(value['scores'])
            
            # Behalte nur die entsprechende Box, Score und Klasse
            new_data[key] = {
                'boxes': value['boxes'][highest_score_idx:highest_score_idx+1],
                'scores': value['scores'][highest_score_idx:highest_score_idx+1],
                'classes': value['classes'][highest_score_idx:highest_score_idx+1]
            }
        else:
            # Falls keine Scores vorhanden sind, füge einen leeren Eintrag hinzu
            new_data[key] = {
                'boxes': np.array([]),
                'scores': np.array([]),
                'classes': np.array([])
            }
    return new_data

# Bereinigtes Dictionary
cleaned_data = keep_highest_score(predictions_yolo1)

ValueError: attempt to get argmax of an empty sequence

In [21]:
len(cleaned_data)

2634

### Crop and Save Images

In [23]:
# Verzeichnisse
image_dir = "D:/Datasets/YOLO_1_Dataset/test/images"   # Ordner mit den Originalbildern
output_dir = "D:/Datasets/Productive_YOLO1/Predicted_Nodules"  # Ordner, wo die gecroppten Bilder gespeichert werden
os.makedirs(output_dir, exist_ok=True)

   
def crop_and_save_image(image_path, boxes, output_name_base):
    # Bild öffnen
    image = Image.open(image_path).convert("RGB")
    
    # Alle Bounding Boxes croppen
    for i, box in enumerate(boxes):
        x_min, y_min, x_max, y_max = map(int, box)  # Koordinaten runden und in int umwandeln
        # print("original",x_min, y_min, x_max, y_max)
        width = x_max - x_min
        height = y_max - y_min
        max_side = max(width, height)
        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2
        x_min = max(0, x_center - max_side / 2)
        y_min = max(0, y_center - max_side / 2)
        x_max = min(image.width, x_center + max_side / 2)
        y_max = min(image.height, y_center + max_side / 2)
        # # Konvertiere zu Ganzzahlen
        x_min, x_max = int(x_min), int(x_max)
        y_min, y_max = int(y_min), int(y_max)
        # print("NEW",x_min, y_min, x_max, y_max)
        cropped_image = image.crop((x_min, y_min, x_max, y_max)) 
        cropped_image = cropped_image.resize((128, 128), Image.LANCZOS) # Cropping
        
        # Speicherpfad für das gecroppte Bild
        cropped_image_path = os.path.join(output_dir, f"{output_name_base}_crop_{i+1}.jpg")
        cropped_image.save(cropped_image_path)
        print(f"Saved cropped image: {cropped_image_path}")

for image_name, info in predictions.items():
    image_path = os.path.join(image_dir, image_name)
    output_name_base = os.path.splitext(image_name)[0]  # Basisname ohne Erweiterung
    
    # Nur croppen und speichern, wenn Bounding Boxes vorhanden sind
    if len(info['boxes']) > 0:
        crop_and_save_image(image_path, info['boxes'], output_name_base)



Saved cropped image: D:/Datasets/Productive_YOLO1/Predicted_Nodules\1.3.6.1.4.1.14519.5.2.1.6655.2359.100048574870269820953143768196_crop_1.jpg
Saved cropped image: D:/Datasets/Productive_YOLO1/Predicted_Nodules\1.3.6.1.4.1.14519.5.2.1.6655.2359.100072635488116034651642445239_crop_1.jpg
Saved cropped image: D:/Datasets/Productive_YOLO1/Predicted_Nodules\1.3.6.1.4.1.14519.5.2.1.6655.2359.100156561188861970258198914981_crop_1.jpg
Saved cropped image: D:/Datasets/Productive_YOLO1/Predicted_Nodules\1.3.6.1.4.1.14519.5.2.1.6655.2359.100423729079097357558203729845_crop_1.jpg
Saved cropped image: D:/Datasets/Productive_YOLO1/Predicted_Nodules\1.3.6.1.4.1.14519.5.2.1.6655.2359.100436650956340012584392599393_crop_1.jpg
Saved cropped image: D:/Datasets/Productive_YOLO1/Predicted_Nodules\1.3.6.1.4.1.14519.5.2.1.6655.2359.100546121805341655431102542094_crop_1.jpg
Saved cropped image: D:/Datasets/Productive_YOLO1/Predicted_Nodules\1.3.6.1.4.1.14519.5.2.1.6655.2359.100711050613761855156496841207_cro

### Save predicted Non-Nodules

In [25]:
import os
import shutil
import numpy as np

# Zielordner erstellen
non_nodule_folder = 'D:/Datasets/Productive_YOLO1/Predicted_Non_Nodule'
os.makedirs(non_nodule_folder, exist_ok=True)

# Verzeichnis mit den Originalbildern
source_directory = 'D:/Datasets/YOLO_1_Dataset/test/images'

# Beispiel-Datenstruktur, die du angegeben hast

# Funktion, um Bilder ohne Boundingbox zu kopieren
def copy_images_without_bboxes(predictions, source_directory, destination_folder):
    for file_name, data in predictions.items():
        # Überprüfen, ob das "boxes" Array leer ist
        if data['boxes'].shape[0] == 0:
            # Bildpfad
            source_image_path = os.path.join(source_directory, file_name)
            
            # Prüfen, ob das Bild existiert
            if os.path.exists(source_image_path):
                # Zielpfad für das Kopieren
                shutil.copy(source_image_path, os.path.join(destination_folder, file_name))
                print(f"Kopiert: {source_image_path} -> {destination_folder}")
            else:
                print(f"Bild nicht gefunden: {file_name}")

# Bilder ohne Boundingbox kopieren
copy_images_without_bboxes(predictions, source_directory, non_nodule_folder)


In [ ]:
import os

folder_path = 'D:/Datasets/yolo1_predicted_non_nodule'

# List all files in the directory
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]


2637

In [ ]:
predicted_non_nodule_labels = 'D:/Datasets/yolo1_predicted_non_nodule_labels'


os.makedirs(predicted_non_nodule_labels, exist_ok=True)


# Verzeichnis, in dem die Originalbilder und Labels gespeichert sind
source_directory = 'D:/Datasets/YOLO_1_Dataset/test/labels'

# Funktion, um die entsprechenden .txt Label-Dateien zu kopieren
def copy_labels(file_list, source_directory, destination_folder):
    for file_name in file_list:
        base_name = os.path.splitext(file_name)[0]  # Ohne Erweiterung
        label_file = base_name + '.txt'  # Label-Datei mit der gleichen Basis

        # Prüfen, ob die Label-Datei existiert
        label_path = os.path.join(source_directory, label_file)
        if os.path.exists(label_path):
            # Kopiere das Label in den Zielordner
            shutil.copy(label_path, os.path.join(destination_folder, label_file))
            print(f"Kopiert: {label_path} -> {destination_folder}")
        else:
            print(f"Label-Datei nicht gefunden für: {file_name}")


# Labels für predicted_nodule kopieren
copy_labels(files, source_directory, predicted_non_nodule_labels)

Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.104507274032170320323347152411.txt -> D:/Datasets/yolo1_predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.104863864417205674007098486529.txt -> D:/Datasets/yolo1_predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.108828997377593078753121028952.txt -> D:/Datasets/yolo1_predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.109466173635014704200062503592.txt -> D:/Datasets/yolo1_predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.109626634592628652877975591041.txt -> D:/Datasets/yolo1_predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.109710522872080626618036223042.txt -> D:/Datasets/yolo1_predicted_non_nodule_labels
Kopi

In [ ]:
# BASE_DIR_TEST = "D:\Datasets"
# seed = 41
# batch_size = 16
# classes = ["yolo1_predicted_non_nodule", "Non_Nodules"]
# image_size = 128
# scenario = 1

# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)

In [ ]:
# test_dataset = CAPS_Productive_Dataset(root_dir=BASE_DIR_TEST, num_images_per_class=len(os.listdir(os.path.join(BASE_DIR_TEST, "Predicted_Nodules"))), classes=classes, scenario=scenario)


## CAPS

In [2]:
BASE_DIR_TEST = "D:/Datasets/Test_Cropped_Images"
seed = 41
batch_size = 16
classes = ["Predicted_Nodules", "Non_Nodules"]
image_size = 128
scenario = 1

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
test_dataset = CAPS_Productive_Dataset(root_dir=BASE_DIR_TEST, num_images_per_class=len(os.listdir(os.path.join(BASE_DIR_TEST, "Predicted_Nodules"))), classes=classes, scenario=scenario)


In [6]:
len(test_dataset)

2765

In [7]:
test_dataset[0]

(<PIL.Image.Image image mode=RGB size=128x128>,
 '1.3.6.1.4.1.14519.5.2.1.6279.6001.479296688495663090411829052528_crop_1.jpg')

In [11]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
test_dataset = TransformDataset(test_dataset, transform=transform)


test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# # Transformiere das cropped_image
# test_dataset = TransformDatasetFinal(cropped_images_dict, transform)




# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [12]:
batch = next(iter(test_loader))
batch

[tensor([[[[-0.1529, -0.1294, -0.1216,  ..., -0.1686, -0.1686, -0.1686],
           [-0.1451, -0.1294, -0.1216,  ..., -0.1608, -0.1608, -0.1529],
           [-0.1451, -0.1373, -0.1294,  ..., -0.1529, -0.1529, -0.1451],
           ...,
           [-0.1137, -0.1137, -0.1137,  ..., -0.0902, -0.0902, -0.0824],
           [-0.1216, -0.1216, -0.1216,  ..., -0.0902, -0.0902, -0.0902],
           [-0.1294, -0.1294, -0.1294,  ..., -0.0980, -0.0902, -0.0902]]],
 
 
         [[[-0.8196, -0.8196, -0.8196,  ..., -0.8980, -0.9137, -0.8980],
           [-0.8275, -0.8275, -0.8275,  ..., -0.9059, -0.9216, -0.9059],
           [-0.8431, -0.8431, -0.8431,  ..., -0.9216, -0.9294, -0.9137],
           ...,
           [-1.0000, -1.0000, -1.0000,  ...,  0.4980,  0.5608,  0.6000],
           [-1.0000, -1.0000, -1.0000,  ...,  0.5529,  0.6157,  0.6627],
           [-1.0000, -1.0000, -1.0000,  ...,  0.5922,  0.6627,  0.7020]]],
 
 
         [[[-0.8275, -0.8353, -0.8431,  ..., -0.1059, -0.0980, -0.0980],
       

In [13]:
TRAIN_ON_GPU = torch.cuda.is_available()
capsule_net = CapsuleNetwork(image_size=image_size, num_classes=len(classes), train_on_gpu=TRAIN_ON_GPU)
checkpoint = torch.load("CAPS_CROPPED/weights/caps_crop.pt")
model_state_dict = checkpoint['model_state_dict']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

capsule_net.load_state_dict(model_state_dict)
capsule_net.to(device)
print(capsule_net)

CapsuleNetwork(
  (conv_layer): ConvLayer(
    (conv): Conv2d(1, 256, kernel_size=(9, 9), stride=(1, 1))
  )
  (primary_capsules): PrimaryCaps(
    (capsules): ModuleList(
      (0-7): 8 x Conv2d(256, 32, kernel_size=(9, 9), stride=(2, 2))
    )
  )
  (digit_capsules): DigitCaps()
  (decoder): Decoder(
    (linear_layers): Sequential(
      (0): Linear(in_features=32, out_features=512, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=512, out_features=1024, bias=True)
      (3): ReLU(inplace=True)
      (4): Linear(in_features=1024, out_features=16384, bias=True)
      (5): Sigmoid()
    )
  )
)


In [14]:
preds,paths = capsule_net.run_model(test_loader)



Batch number:  0
Batch number:  1
Batch number:  2
Batch number:  3
Batch number:  4
Batch number:  5
Batch number:  6
Batch number:  7
Batch number:  8
Batch number:  9
Batch number:  10
Batch number:  11
Batch number:  12
Batch number:  13
Batch number:  14
Batch number:  15
Batch number:  16
Batch number:  17
Batch number:  18
Batch number:  19
Batch number:  20
Batch number:  21
Batch number:  22
Batch number:  23
Batch number:  24
Batch number:  25
Batch number:  26
Batch number:  27
Batch number:  28
Batch number:  29
Batch number:  30
Batch number:  31
Batch number:  32
Batch number:  33
Batch number:  34
Batch number:  35
Batch number:  36
Batch number:  37
Batch number:  38
Batch number:  39
Batch number:  40
Batch number:  41
Batch number:  42
Batch number:  43
Batch number:  44
Batch number:  45
Batch number:  46
Batch number:  47
Batch number:  48
Batch number:  49
Batch number:  50
Batch number:  51
Batch number:  52
Batch number:  53
Batch number:  54
Batch number:  55
Ba

In [19]:
# Anzahl der 1er zählen
num_ones = preds.count(1)

# Anzahl der 0er zählen
num_zeros = preds.count(0)

print("Anzahl der 1er:", num_ones)
print("Anzahl der 0er:", num_zeros)

Anzahl der 1er: 105
Anzahl der 0er: 2660


In [20]:
torch.save(preds,"preds_caps.pt")

In [21]:
torch.save(paths,"paths_caps.pt")

In [26]:
# Kürzen der Filenames
shortened_paths = [
    path.split('_', 1)[0] + '.' + path.split('.')[-1] for path in paths
]

In [31]:
# Listen für Klassen erstellen
class_1_files = [shortened_paths[i] for i in range(len(shortened_paths)) if preds[i] == 1]
class_0_files = [shortened_paths[i] for i in range(len(shortened_paths)) if preds[i] == 0]

print("Filenames mit Klasse 1:", len(class_1_files))
print("Filenames mit Klasse 0:", len(class_0_files))

Filenames mit Klasse 1: 105
Filenames mit Klasse 0: 2660


In [39]:
# Entfernen der Dateien, die auch in class_0_files vorhanden sind
class_1_files_cleaned = [file for file in class_1_files if file not in class_0_files]
len(class_1_files_cleaned)

100

In [43]:
# Nur eindeutige Werte
unique_class_1_files = pd.unique(class_1_files_cleaned)
unique_class_1_files=unique_class_1_files.tolist()
len(unique_class_1_files)

C:\Users\chris\AppData\Local\Temp\ipykernel_23352\802581285.py:2: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_class_1_files = pd.unique(class_1_files_cleaned)


99

In [44]:
# Nur eindeutige Werte
unique_class_0_files = pd.unique(class_0_files)
unique_class_0_files=unique_class_0_files.tolist()
len(unique_class_0_files)

C:\Users\chris\AppData\Local\Temp\ipykernel_23352\2467106232.py:2: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_class_0_files = pd.unique(class_0_files)


2634

In [47]:
# Ordner erstellen
class_0_folder = 'D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule'
class_1_folder = 'D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule'

os.makedirs(class_0_folder, exist_ok=True)
os.makedirs(class_1_folder, exist_ok=True)

# Verzeichnis, in dem die Originalbilder gespeichert sind
source_directory = 'D:/Datasets/YOLO_1_Dataset/test/images'

# Funktion, um Dateien mit anderen Erweiterungen zu finden und zu kopieren
def copy_files(file_list, destination_folder):
    for file_name in file_list:
        # Datei ohne die Erweiterung (nur der Name)
        base_name = os.path.splitext(file_name)[0]

        # Suche nach der Datei mit verschiedenen Erweiterungen
        for extension in ['.jpg', '.jpeg', '.JPEG']:
            source_file = os.path.join(source_directory, base_name + extension)
            if os.path.exists(source_file):
                # Datei kopieren
                shutil.copy(source_file, os.path.join(destination_folder, base_name + extension))
                print(f"Kopiert: {source_file} -> {destination_folder}")
                break
        else:
            print(f"Datei nicht gefunden: {file_name}")



In [49]:
copy_files(unique_class_0_files, class_0_folder)


Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.100048574870269820953143768196.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.100072635488116034651642445239.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.100156561188861970258198914981.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.100423729079097357558203729845.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.100436650956340012584392599393.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.100546121805341655431102542094.jpeg -> D:/D

In [48]:

copy_files(unique_class_1_files, class_1_folder)

Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6279.6001.479296688495663090411829052528.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.102327112232649985044931869364.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.104495950603684654013760022915.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.105311559724535680995430937204.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.105800557057336206791850195590.jpeg -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule
Kopiert: D:/Datasets/YOLO_1_Dataset/test/images\1.3.6.1.4.1.14519.5.2.1.6655.2359.10679969569579182379810

In [50]:
import os
import shutil


predicted_nodule_labels = 'D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule_labels'
predicted_non_nodule_labels = 'D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule_labels'
  # Ordner für Labels


os.makedirs(predicted_nodule_labels, exist_ok=True)
os.makedirs(predicted_non_nodule_labels, exist_ok=True)


# Verzeichnis, in dem die Originalbilder und Labels gespeichert sind
source_directory = 'D:/Datasets/YOLO_1_Dataset/test/labels'

# Funktion, um die entsprechenden .txt Label-Dateien zu kopieren
def copy_labels(file_list, source_directory, destination_folder):
    for file_name in file_list:
        base_name = os.path.splitext(file_name)[0]  # Ohne Erweiterung
        label_file = base_name + '.txt'  # Label-Datei mit der gleichen Basis

        # Prüfen, ob die Label-Datei existiert
        label_path = os.path.join(source_directory, label_file)
        if os.path.exists(label_path):
            # Kopiere das Label in den Zielordner
            shutil.copy(label_path, os.path.join(destination_folder, label_file))
            print(f"Kopiert: {label_path} -> {destination_folder}")
        else:
            print(f"Label-Datei nicht gefunden für: {file_name}")


# Labels für predicted_nodule kopieren
copy_labels(unique_class_0_files, source_directory, predicted_nodule_labels)

Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.100048574870269820953143768196.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.100072635488116034651642445239.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.100156561188861970258198914981.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.100423729079097357558203729845.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.100436650956340012584392599393.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.1005461218053

In [51]:

# Labels für predicted_nodule kopieren
copy_labels(unique_class_1_files, source_directory, predicted_non_nodule_labels)


Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.479296688495663090411829052528.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.102327112232649985044931869364.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.104495950603684654013760022915.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.105311559724535680995430937204.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.6655.2359.105800557057336206791850195590.txt -> D:/Datasets/YOLO_2_Dataset_produktiv/predicted_non_nodule_labels
Kopiert: D:/Datasets/YOLO_1_Dataset/test/labels\1.3.6.1.4.1.14519.5.2.1.665

## YOLO2

In [6]:
model=YOLO("YOLO1/1/weights/best.pt")
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_

In [7]:
image_folder="D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule"
label_folder="D:/Datasets/YOLO_2_Dataset_produktiv/predicted_nodule_labels"

test_images= [f for f in os.listdir(image_folder) if f.endswith((".jpeg", ".JPEG"))]
test_labels= [f for f in os.listdir(label_folder) if f.endswith((".txt"))]
print(f"Gefundene testbilder: {len(test_images)}")
print(f"Gefundene testlabels: {len(test_labels)}")
print(os.path.splitext(test_images[0])[0])
image_bbox_pairs = [
                    (os.path.join(image_folder, img), os.path.join(label_folder,img.replace(".jpeg",".txt").replace(".JPEG",".txt")))
                    for img in  test_images
                    if img.replace(".jpeg",".txt").replace(".JPEG",".txt") in test_labels
]
print(f"Gefundene Paare: {len(image_bbox_pairs)}")

Gefundene testbilder: 2634
Gefundene testlabels: 2634
1.3.6.1.4.1.14519.5.2.1.6655.2359.100048574870269820953143768196
Gefundene Paare: 2634


In [9]:
def load_ground_truth(file_path, image_shape):
    h,w = image_shape[:2]
    boxes=[]

    with open(file_path, "r") as f:
        for line in f.readlines():
            class_id,x_center, y_center, width, height = map(float, line.strip().split())

            x_min = int((x_center - width/2)*w)
            y_min = int((y_center - width/2)*h)
            x_max = int((x_center + width/2)*w)
            y_max = int((y_center + width/2)*h)
        
            boxes.append([x_min,y_min,x_max,y_max])
    return boxes

In [10]:
predictions={}
ground_truths={}

for image_path, bbox_path in image_bbox_pairs:
    image = Image.open(image_path)

    image_np = np.array(image)

    gt_boxes = load_ground_truth(bbox_path, image_np.shape)
    ground_truths[os.path.basename(image_path)] = gt_boxes

    results = model(image_np)

    boxes = results[0].boxes.xyxy.cpu().numpy()
    scores = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()

    predictions[os.path.basename(image_path)]= {
        "boxes": boxes,
        "scores": scores,
        "classes": classes
    }


0: 512x512 1 class1, 8.0ms
Speed: 3.0ms preprocess, 8.0ms inference, 61.9ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 0.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 class1, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512

In [15]:
predictions

{'1.3.6.1.4.1.14519.5.2.1.6655.2359.100048574870269820953143768196.jpeg': {'boxes': array([[     337.79,      298.65,      360.08,      322.51]], dtype=float32),
  'scores': array([    0.68575], dtype=float32),
  'classes': array([          0], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.100072635488116034651642445239.jpeg': {'boxes': array([[     143.24,       274.3,      203.74,      338.64]], dtype=float32),
  'scores': array([    0.79879], dtype=float32),
  'classes': array([          0], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.100156561188861970258198914981.jpeg': {'boxes': array([[     123.94,      288.28,      178.24,      353.88]], dtype=float32),
  'scores': array([    0.80081], dtype=float32),
  'classes': array([          0], dtype=float32)},
 '1.3.6.1.4.1.14519.5.2.1.6655.2359.100423729079097357558203729845.jpeg': {'boxes': array([[     312.01,      288.82,      393.97,      356.54]], dtype=float32),
  'scores': array([    0.79611], dtype=float32),
  '

In [17]:
def compute_iou(box1, box2):
    """
    Berechnet Intersection over Union (IoU) für zwei Bounding Boxes.
    Boxformat: [x_min, y_min, x_max, y_max]
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    # Flächen von Intersection und Union berechnen
    intersection = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union = box1_area + box2_area - intersection
    
    return intersection / union if union > 0 else 0

def evaluate(predictions, ground_truths, iou_threshold=0.5, score_threshold=0.25):
    """
    Evaluierung der Vorhersagen im Vergleich zu den Ground Truths.
    predictions: Dictionary mit Vorhersagen
    ground_truths: Dictionary mit Ground Truths
    """
    tp, fp, fn, tn = 0, 0, 0, 0

    for image_id, gt_boxes in ground_truths.items():
        pred_boxes = predictions[image_id]['boxes']
        pred_scores = predictions[image_id]['scores']
        
        # Filter Vorhersagen basierend auf Score Threshold
        # pred_boxes = pred_boxes[pred_scores >= score_threshold]
        
        # Fall: Keine Ground Truth und keine Vorhersagen
        if len(gt_boxes) == 0 and len(pred_boxes) == 0:
            tn += 1
            continue

        matched_gt = set()
        for pred_box in pred_boxes:
            # Berechne IoU mit allen Ground Truths
            ious = [compute_iou(pred_box, gt_box) for gt_box in gt_boxes]
            max_iou = max(ious) if ious else 0  # maximaler IoU-Wert für diese Vorhersage
            
            if max_iou >= iou_threshold:
                # True Positive: Eine Ground Truth wurde erkannt
                matched_gt.add(ious.index(max_iou))
            for iou in ious:
                if iou >= iou_threshold:
                    # True Positive: Eine Ground Truth wurde erkann
                    tp += 1
                else:
                    # False Positive: Keine passende Ground Truth gefunden
                    fp += 1

        
        # False Negatives: Nicht erkannte Ground Truths
        fn += len(gt_boxes) - len(matched_gt)

    # Metriken berechnen
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    accuracy = (tp + 2000) / (tp + fp + fn + 2000) if (tp + fp + fn + tn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy,
        "specificity": specificity,
        "f1_score": f1_score,
        "tp": tp,
        "fp": fp,
        "fn": fn,
        "tn": tn,
    }


results = evaluate(predictions, ground_truths)
print(results)


{'precision': 0.9260252970486776, 'recall': 0.934261407579273, 'accuracy': 0.9240426867545511, 'specificity': 0.0, 'f1_score': 0.9301251203079884, 'tp': 2416, 'fp': 193, 'fn': 170, 'tn': 0}


In [ ]:
import os
import shutil
import numpy as np

# Zielordner erstellen
non_nodule_folder = 'D:/Datasets/yolo1_predicted_non_nodule'
os.makedirs(non_nodule_folder, exist_ok=True)

# Verzeichnis mit den Originalbildern
source_directory = 'D:/Datasets/YOLO_1_Dataset/test/images'

# Beispiel-Datenstruktur, die du angegeben hast

# Funktion, um Bilder ohne Boundingbox zu kopieren
def copy_images_without_bboxes(predictions, source_directory, destination_folder):
    for file_name, data in predictions.items():
        # Überprüfen, ob das "boxes" Array leer ist
        if data['boxes'].shape[0] == 0:
            # Bildpfad
            source_image_path = os.path.join(source_directory, file_name)
            
            # Prüfen, ob das Bild existiert
            if os.path.exists(source_image_path):
                # Zielpfad für das Kopieren
                shutil.copy(source_image_path, os.path.join(destination_folder, file_name))
                print(f"Kopiert: {source_image_path} -> {destination_folder}")
            else:
                print(f"Bild nicht gefunden: {file_name}")

# Bilder ohne Boundingbox kopieren
copy_images_without_bboxes(predictions, source_directory, non_nodule_folder)